In [1]:
! pip install beautifulsoup4
! pip install lxml
! pip install geocoder
# 
! pip install geopy
! pip install folium

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
     |████████████████████████████████| 5.8MB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 2.6MB/s ta 0:00:011
     |████████████████████████████████| 829kB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
     |████████████████████████████████| 102kB 2.8MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


In [2]:
! pip install requests
! pip install html5lib
! pip install bs4

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


## Notebook to scrape the Wikipedia page <a href=https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>List of postal codes of Canada: M</a>

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

In [23]:
from bs4 import BeautifulSoup

In [24]:
import folium

In [25]:
from geopy.geocoders import Nominatim

# States of ( India) from web

In [26]:
url = "https://www.indianmirror.com/india-post/indianpincode.html"

In [27]:
import bs4
from urllib.request import urlopen

In [28]:
html = urlopen(url)

In [29]:
soup = BeautifulSoup(html, 'html.parser')

### To perform web scraping importing the libraries.
The urllib.request module is used to open URLs.
The Beautiful Soup package is used to extract data from html files.
The Beautiful Soup library's name is bs4 which stands for Beautiful Soup version 4.

In [30]:
title = soup.title
print(title)

<title>Indian Pincode, Area code, List of Districts in India, Indian Districts </title>


In [31]:
addr = 'Bangalore, Karnataka'
geoloc = Nominatim(user_agent="bangalore_explorer")
location = geoloc.geocode(addr)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [32]:
import requests
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

In [33]:
df.shape

(23, 6)

In [34]:
df.head()

,0,1,2,3,4,5
0,States,States,States,States,States,States
1,S.No,State,Dist,S.No,State,Dist
2,1,Andhra Pradesh,23,15,Maharashtra,35
3,2,Arunachal Pradesh,16,16,Manipur,9
4,3,Assam,27,17,Meghalaya,7


In [35]:
## Import geodata from CSV file
geodata = "India_pin_latlon.csv"
df_geodata = pd.read_csv(geodata)
df_geodata.head()

,country,pincode,locality,state,district,city,latitude,longitude
0,IN,744101,Marine Jetty,Andaman & Nicobar Islands,South Andaman,Portblair,11.6667,92.7500
1,IN,744101,Port Blair,Andaman & Nicobar Islands,South Andaman,Port Blair,11.6667,92.7500
2,IN,744101,N.S.Building,Andaman & Nicobar Islands,South Andaman,Portblair,11.6667,92.7500
3,IN,744102,Haddo,Andaman & Nicobar Islands,South Andaman,Port Blair,11.6833,92.7167
4,IN,744102,Chatham,Andaman & Nicobar Islands,South Andaman,Portblair,11.7000,92.6667


In [36]:
df_main = df_geodata[df_geodata.district == 'Bangalore']

In [37]:
df_main.head()

,country,pincode,locality,state,district,city,latitude,longitude
50019,IN,560001,Bangalore G.P.O.,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
50020,IN,560001,Legislators Home,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
50021,IN,560001,Vasanthanagar,Karnataka,Bangalore,NaN,12.9914,77.5944
50022,IN,560001,Mahatma Gandhi Road,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
50023,IN,560001,Vidhana Soudha,Karnataka,Bangalore,Bangalore North,12.9914,77.5944


In [38]:
df_main.shape

(278, 8)

In [39]:
LIMIT = 100
CLIENT_ID = 'HEUOIKGRIHN0N42H4SF0QD15XHFYVAJHQ3D2DHVZBRCUST0S'
CLIENT_SECRET = 'ENEK5AYTY4XA4FCHBQQM5ZHFZNTRD1SCM5VPXGNHIMXDUBQN'
VERSION = '20180604'
# Create a new function
def getVenues(names, latitudes, longitudes, radius=500):
    
    list_of_venues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        list_of_venues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    venues = pd.DataFrame([item for venue_list in list_of_venues for item in venue_list])
    venues.columns = ['locality', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(venues)

In [41]:
venues_of_bangalore = getVenues(names=df_main['locality'], 
                              latitudes=df_main['latitude'],
                              longitudes=df_main['longitude']
                              )

Bangalore G.P.O.
Legislators Home
Vasanthanagar
Mahatma Gandhi Road
Vidhana Soudha
Rajbhavan (Bangalore)
HighCourt
Cubban Road
Bangalore Bazaar
CMM Court Complex
Dr. Ambedkar Veedhi
Bangalore City
Bangalore Corporation Building
Sri Jayachamarajendra Road
Palace Guttahalli
Malleswaram
Swimming Pool Extn
Vyalikaval Extn
Pampamahakavi Road
Thyagarajnagar
Mavalli
Basavanagudi
Fraser Town
J.C.Nagar
Training Command IAF
Agram
Air Force Hospital
Hulsur Bazaar
H.A.L II Stage
K. G. Road
Bangalore Dist Offices Bldg
Rajajinagar
Industrial Estate (Bangalore)
Rajajinagar IVth Block
Jayangar III Block
Madhavan Park
Science Institute
Jalahalli
Jalahalli East
Jalahalli West
Krishnarajapuram R S
Doorvaninagar
Ramamurthy Nagar
Vimanapura
NAL
Chamrajpet (Bangalore)
Gaviopuram Extension
Seshadripuram
Gayathrinagar
Srirampuram
Yeshwanthpur Bazar
Yeswanthpura
Magadi Road
Anandnagar (Bangalore)
Hebbal Kempapura
H.A. Farm
Museum Road
Richmond Town
CMP Centre And School
Deepanjalinagar
Governmemnt Electric Fac

In [42]:
print(venues_of_bangalore.shape)
venues_of_bangalore.head()

(3126, 7)


,locality,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bangalore G.P.O.,12.9914,77.5944,Millers 46,12.991666,77.594207,Steakhouse
1,Bangalore G.P.O.,12.9914,77.5944,Ujwal Bar & Restaurant,12.992280,77.594473,Indian Restaurant
2,Bangalore G.P.O.,12.9914,77.5944,Alliance Française,12.991232,77.596723,Concert Hall
3,Bangalore G.P.O.,12.9914,77.5944,Infinitea,12.987157,77.594835,Tea Room
4,Bangalore G.P.O.,12.9914,77.5944,Imperial Restaurant,12.991150,77.593837,Indian Restaurant


### Now encode the data

In [43]:
trans_data = pd.get_dummies(venues_of_bangalore[['Venue Category']], prefix="", prefix_sep="")
# add neighbourhood column back to dataframe
trans_data['locality'] = venues_of_bangalore['locality'] 
# move neighbourhood column to the first
fixed_columns = [trans_data.columns[-1]] + list(trans_data.columns[:-1])
trans_data = trans_data[fixed_columns]
trans_data.head()

,locality,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,...,Tourist Information Center,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Bangalore G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bangalore G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bangalore G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bangalore G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bangalore G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Now group rows by neighbourhood taking mean of the frequency of occurrence of each category

In [44]:
group_by_venue = trans_data.groupby('locality').mean().reset_index()
group_by_venue.head()

,locality,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,...,Tourist Information Center,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,A F Station Yelahanka,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Adugodi,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,Agara,0.0,0.022727,0.022727,0.0,0.0,0.0,0.0,0.0,0.022727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022727,0.0
3,Amruthahalli,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,Anandnagar (Bangalore),0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


### Let's print each neighbourhood along with top 5 common venues

In [45]:
topVenues = 5

for v in group_by_venue['locality']:
    print("----"+v+"----")
    temp = group_by_venue[group_by_venue['locality'] == v].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(topVenues))
    print('\n')

----A F Station Yelahanka----
                 venue  freq
0          IT Services   1.0
1  American Restaurant   0.0
2      Motorcycle Shop   0.0
3        Movie Theater   0.0
4            Multiplex   0.0


----Adugodi----
                           venue  freq
0              Indian Restaurant  0.25
1                    Bus Station  0.25
2                  Design Studio  0.25
3           Fast Food Restaurant  0.25
4  Paper / Office Supplies Store  0.00


----Agara----
               venue  freq
0               Café  0.20
1  Indian Restaurant  0.11
2        Coffee Shop  0.07
3     Ice Cream Shop  0.07
4                Pub  0.05


----Amruthahalli----
                   venue  freq
0                 Resort  0.12
1     Italian Restaurant  0.12
2  Hyderabadi Restaurant  0.12
3      Indian Sweet Shop  0.12
4   Fast Food Restaurant  0.12


----Anandnagar (Bangalore)----
                        venue  freq
0  Tourist Information Center   1.0
1                         ATM   0.0
2          Misce

## Function to sort the venues in descending order

In [46]:
def common_venues(venue_var, top_how_many):
    venue_cat = venue_var.iloc[1:]
    sorted_venue_cat = venue_cat.sort_values(ascending=False)
    
    return sorted_venue_cat.index.values[0:top_how_many]

## Now create a new dataframe and display the top 10 venues for each neighborhood.

In [47]:
topVenues = 10

indicators = ['st', 'nd', 'rd']
# create columns against top venues
column_names = ['locality']
for ind in np.arange(topVenues):
    try:
        column_names.append('{}{} Most Frequent Venue'.format(ind+1, indicators[ind]))
    except:
        column_names.append('{}th Most Frequent Venue'.format(ind+1))

# create a new dataframe of sorted venues
sorted_v = pd.DataFrame(columns=column_names)
sorted_v['locality'] = group_by_venue['locality']

for ind in np.arange(group_by_venue.shape[0]):
    sorted_v.iloc[ind, 1:] = common_venues(group_by_venue.iloc[ind, :], topVenues)

sorted_v.head()

,locality,1st Most Frequent Venue,2nd Most Frequent Venue,3rd Most Frequent Venue,4th Most Frequent Venue,5th Most Frequent Venue,6th Most Frequent Venue,7th Most Frequent Venue,8th Most Frequent Venue,9th Most Frequent Venue,10th Most Frequent Venue
0,A F Station Yelahanka,IT Services,Hotel,Fishing Store,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Donut Shop,Dive Bar,Diner
1,Adugodi,Fast Food Restaurant,Indian Restaurant,Bus Station,Design Studio,Zoo,Dive Bar,Farmers Market,Event Service,Electronics Store,Donut Shop
2,Agara,Café,Indian Restaurant,Ice Cream Shop,Coffee Shop,Pub,Lounge,Bar,Mexican Restaurant,Brewery,Breakfast Spot
3,Amruthahalli,Hyderabadi Restaurant,Indian Sweet Shop,Café,Italian Restaurant,Resort,Badminton Court,Coffee Shop,Fast Food Restaurant,Deli / Bodega,Department Store
4,Anandnagar (Bangalore),Tourist Information Center,Flea Market,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Donut Shop,Dive Bar,Diner,Zoo


### Run k-means to cluster the neighbourhoods

In [48]:
# number of clusters to made
no_c = 5

venue_clusters = group_by_venue.drop('locality', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=no_c, random_state=0).fit(venue_clusters)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 4, 4, 3, 1, 4, 4, 4, 1], dtype=int32)

In [49]:
df_main.head()

,country,pincode,locality,state,district,city,latitude,longitude
50019,IN,560001,Bangalore G.P.O.,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
50020,IN,560001,Legislators Home,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
50021,IN,560001,Vasanthanagar,Karnataka,Bangalore,NaN,12.9914,77.5944
50022,IN,560001,Mahatma Gandhi Road,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
50023,IN,560001,Vidhana Soudha,Karnataka,Bangalore,Bangalore North,12.9914,77.5944


In [50]:
# add cluster labels
sorted_v.insert(0, 'Clusters', kmeans.labels_)

In [51]:
blr_df = df_main

In [52]:
sorted_v.head()

,Clusters,locality,1st Most Frequent Venue,2nd Most Frequent Venue,3rd Most Frequent Venue,4th Most Frequent Venue,5th Most Frequent Venue,6th Most Frequent Venue,7th Most Frequent Venue,8th Most Frequent Venue,9th Most Frequent Venue,10th Most Frequent Venue
0,2,A F Station Yelahanka,IT Services,Hotel,Fishing Store,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Donut Shop,Dive Bar,Diner
1,4,Adugodi,Fast Food Restaurant,Indian Restaurant,Bus Station,Design Studio,Zoo,Dive Bar,Farmers Market,Event Service,Electronics Store,Donut Shop
2,4,Agara,Café,Indian Restaurant,Ice Cream Shop,Coffee Shop,Pub,Lounge,Bar,Mexican Restaurant,Brewery,Breakfast Spot
3,4,Amruthahalli,Hyderabadi Restaurant,Indian Sweet Shop,Café,Italian Restaurant,Resort,Badminton Court,Coffee Shop,Fast Food Restaurant,Deli / Bodega,Department Store
4,3,Anandnagar (Bangalore),Tourist Information Center,Flea Market,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Donut Shop,Dive Bar,Diner,Zoo


In [53]:
# merge main dataframe with previous dataframe to add lat long for the neighbourhoods
blr_df = blr_df.join(sorted_v.set_index('locality'), on='locality')
blr_df = blr_df.fillna(0)
blr_df["Clusters"] = blr_df["Clusters"].astype(int)
blr_df.head()

,country,pincode,locality,state,district,city,latitude,longitude,Clusters,1st Most Frequent Venue,2nd Most Frequent Venue,3rd Most Frequent Venue,4th Most Frequent Venue,5th Most Frequent Venue,6th Most Frequent Venue,7th Most Frequent Venue,8th Most Frequent Venue,9th Most Frequent Venue,10th Most Frequent Venue
50019,IN,560001,Bangalore G.P.O.,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Chinese Restaurant,Italian Restaurant,Tea Room,Steakhouse,Hotel,Juice Bar,Fast Food Restaurant,Concert Hall
50020,IN,560001,Legislators Home,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Chinese Restaurant,Italian Restaurant,Tea Room,Steakhouse,Hotel,Juice Bar,Fast Food Restaurant,Concert Hall
50021,IN,560001,Vasanthanagar,Karnataka,Bangalore,0,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Chinese Restaurant,Italian Restaurant,Tea Room,Steakhouse,Hotel,Juice Bar,Fast Food Restaurant,Concert Hall
50022,IN,560001,Mahatma Gandhi Road,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Chinese Restaurant,Italian Restaurant,Tea Room,Steakhouse,Hotel,Juice Bar,Fast Food Restaurant,Concert Hall
50023,IN,560001,Vidhana Soudha,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Chinese Restaurant,Italian Restaurant,Tea Room,Steakhouse,Hotel,Juice Bar,Fast Food Restaurant,Concert Hall


### Visualize the clusters

In [54]:
# create map
cl_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(no_c)
ys = [i + x + (i*x)**2 for i in range(no_c)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_df['latitude'], blr_df['longitude'], blr_df['locality'], blr_df['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cl_map)
       
cl_map